# pyUserCalc

This is a python jupyter notebook that calculates uranium-series disequilibrium activity ratios in partial melts, for a 1D mantle decompression melting scenario. The notebook can determine U-series activities in partial melts for either of two end-member transport models.

One model option will determine melt compositions for a full equilibrium transport scenario (reactive porous flow) by recreating the functionality of Spiegelman's [UserCalc](http://www.ldeo.columbia.edu/~mspieg/UserCalc) website for calculating Uranium-series disequilibria for the equilbrium transport model described in:

    Spiegelman, M., 2000. UserCalc: A Web-based uranium series calculator for magma migration problems. Geochem. Geophys. Geosyst. 1, 1016. https://doi.org/10.1029/1999GC000030.

The other model determines melt compositions for a full disequilibrium transport scenario (i.e., there is no melt-rock reequilibration after melt generation, but aggregated melts are still transported along the length of the melt column), using the disequilibrium transport model described in the Appendix of Spiegelman, M. and Elliott T. (1993).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Enter initial input information

In [ ]:
# The user should set initial inputs by editing the code in this cell.

# Set the name for the input data file to be used in this model run.
# The name in quotes below should match the name of your input file,
# minus the file extension (.csv). You may download the sample file 
# to see the correct format for user input files. Your input file
# should be saved to the "data" folder prior to running this code.

runname='sample'

# Set the final melting pressure for this model run. This pressure will
# be used to impose a lithospheric cap that truncates the melt column
# and terminates the melting calculation. Note that any transport time after
# melting terminates is not automatically calculated by this model.

Plithos = 5.0  # kilobars

## Some plotting utilities

In [ ]:
# This cell defines a series of plots that the user may wish to see as output after
# running the model.

import matplotlib as mpl
mpl.rcParams['lines.linewidth']=2
mpl.rcParams['font.size']=16

def plot_inputs(df,figsize=(8,6)):
    ''' 
        pretty plots input data from pandas dataframe df
    '''
    
    fig, (ax1, ax2, ax3) = plt.subplots(1,3, sharey=True,figsize=figsize)
    ax1.plot(df['F'],df['P'])
    ax1.invert_yaxis()
    ax1.set_xlabel('F')
    ax1.set_ylabel('Pressure (kb)')
    xticks = np.linspace(0,max(df['F']),10)
    ax1.grid()
    ax1.set_xticks(xticks,minor=True)
    ax2.plot(df['Kr'],df['P'])
    ax2.set_xlabel('Kr')
    for s in ['DU','DTh','DRa','DPa']:
        ax3.semilogx(df[s],df['P'],label=s)
    ax3.set_xlabel('Ds')
    ax3.legend(loc='best',bbox_to_anchor=(1.1,1))
    
def plot_1Dcolumn(df,figsize=(8,6)):
    '''
        pretty plots output data from dataframe of output
    '''
    
    fig, (ax1, ax2) = plt.subplots(1,2, sharey=True,figsize=figsize)
    ax1.plot(df['phi'],df['P'],'r',label='$\phi$')
    ax1.set_xlabel('Porosity',color='r')
    ax1.set_ylabel('Pressure (kb)')
    ax1.invert_yaxis()

    ax1a = ax1.twiny()
    ax1a.plot(df['F'],df['P'],'b',label='$F$')
    ax1a.set_xlabel('Degree of melting',color='b')

    for s in ['(230Th/238U)','(226Ra/230Th)','(231Pa/235U)']:
        ax2.plot(df[s],df['P'],label=s)
    ax2.set_xlabel('Activity Ratios')
    ax2.set_xlim(0,5)
    ax2.set_xticks(range(5))
    ax2.grid()
    ax2.legend(loc='best',bbox_to_anchor=(1.1,1))
    return fig,(ax1,ax1a,ax2)

def plot_contours(phi0,W0,act,figsize=(8,8)):
    '''
    pretty plot activity contour plots
    '''
    
    Nplots = act.shape[0]
    if Nplots == 2:
        labels = ['(230Th/238U)', '(226Ra/230Th)']
    else:
        labels = ['(231Pa/235U)']
    
    
    if Nplots == 1:
        plt.figure(figsize=figsize)
        cf = plt.contourf(phi0,W0,act[0])
        plt.xscale('log')
        plt.yscale('log')
        plt.xlabel('Porosity ($\phi_0$)')
        plt.ylabel('Upwelling Rate (cm/yr)')
        plt.gca().set_aspect('equal')
        plt.title(labels[0])
        plt.colorbar(cf, ax=plt.gca(), orientation='horizontal',shrink=1.)
    else:
        fig, axes = plt.subplots(1,Nplots,sharey=True,figsize=figsize)
        for i,ax in enumerate(axes):
            cf = ax.contourf(phi0,W0,act[i])
            ax.set_xscale('log')
            ax.set_yscale('log')
            ax.set_aspect('equal')
            ax.set_xlabel('Porosity ($\phi_0$)')
            ax.set_ylabel('Upwelling Rate (cm/yr)')
            ax.set_title(labels[i])
            fig.colorbar(cf, ax=ax, orientation='horizontal',shrink=1.)

def plot_mesh_Ra(Th,Ra,figsize=(8,8)):
    '''
    activity mesh plot for Ra vs. Th, from gridded data
    '''    
    mW,nphi=Th.shape    
    plt.figure()
    for m in range(mW):
        plt.plot(Th[m],Ra[m],label='W = {} cm/yr'.format(W0[m]))
    for n in range(nphi):
        plt.plot(Th.T[n],Ra.T[n],label='$\phi$ = {}'.format(phi0[n]))

    plt.legend(loc='best',bbox_to_anchor=(1.1,1))
    plt.axis([0.9,1.5,0.7,4.0])
    plt.xlabel('($^{230}$Th/$^{238}$U)')
    plt.ylabel('($^{226}$Ra/$^{230}$Th)')
#    plt.axes().set_aspect('equal',)

    plt.savefig("{}_Ra-Th.ps".format(runname))

def plot_mesh_Pa(Th,Pa,figsize=(8,8)):
    '''
    activity mesh plot for Pa vs. Th, from gridded data
    '''
    mW,nphi = Th.shape
    plt.figure()
    for m in range(mW):
        plt.plot(Th[m],Pa[m],label='W = {} cm/yr'.format(W0[m]))
    for n in range(nphi):
        plt.plot(Th.T[n],Pa.T[n],label='$\phi$ = {}'.format(phi0[n]))

#    plt.legend(loc='best',bbox_to_anchor=(1.1,1))
    plt.axis([0.9,1.5,0.7,4.0])
    plt.xlabel('($^{230}$Th/$^{238}$U)')
    plt.ylabel('($^{226}$Ra/$^{230}$Th)')
    
    plt.savefig("{}_Pa-Th.ps".format(runname))
    
    plt.show()


## An equilibrium decay chain solver class

This class solves the generic radiocative decay chain problem for the logarithm of the radiogenic component of melt concentration, after Equation 9 in Spiegelman (2000).

$$
    \frac{dU_i^r}{dz} = \lambda'_i\frac{\overline{\rho D_{i}}}{\overline{F D_{i}}}\left[ R_i^{i-1}\exp[U_{i-1}-U_i] - 1\right]
$$

where 
$$
    \lambda'_i = \frac{h\lambda_i}{W_0}
$$ 

are the decay constants scaled by the solid transport time ($h/W_0$) across a layer of depth $h$.

\begin{align}
    \overline{\rho D_{i}} &= \frac{\rho_f}{\rho_s}\phi + (1 -\phi)D_i\\
    \overline{F D_{i}} &= F + (1 -F)D_i
\end{align}

and

$$
    R_i^{i-1} = \alpha_i\frac{D_i^0}{D_{i-1}^0}\frac{\overline{\rho D_{i-1}}}{\overline{\rho D_{i}}}
$$

is the ingrowth factor. $\alpha$ is the initial degree of secular disequilibrium in the *unmelted solid*.

Note:  $U_i = \log(c_f/c_f^0)$ is the total log of the concentration of nuclide $i$ in the melt which can be decomposed into

$$
  U_i = U^s_i + U^r_i
$$

where

$$
    U^s_i = \log\left[ \frac{D_i^0}{\overline{FD}_i}\right]
$$

is the log concentration of a stable nuclide with the same partition coefficients.  U^r_i is the radiogenic ingrowth component.

In [ ]:
# This is the equilibrium transport calculator. You probably will not need or want to edit this cell.
# A possible exception: the ODE solver method can be changed if desired, but the default is probably best.

from scipy.integrate import solve_ivp

class EquilTransport:
    '''
    A class for calculating generic radioactive decay chains for the scaled equations 9 in Spiegelman (2000)
    
    Usage:  solver=model(alpha0,lambdas,D,F,dFdz,phi,rho_f=2800.,rho_s=3300.,method='Radau')
    
    Inputs:
        alpha0  :  numpy array of initial activities
        lambdas :  decay constants scaled by solid transport time
        D       :  Function returning an array of partition coefficents at scaled height z'
        F       :  Function that returns the degree of melting F as a function of  z'
        dFdz    :  Function that returns the derivative of F with respect to z'
        phi     :  Function that returns the porosity phi as a function of z'
        rho_f   :  melt_density
        rho_s   :  solid_density
        method  :  ODE solver method to be passed to ode_solveivp (one of 'RK45', 'Radau', 'BDF')
        
    Outputs:  pandas DataFrame with columns z, Us, Uf
    '''      
    def __init__(self,alpha0,lambdas,D,F,dFdz,phi,rho_f=2800., rho_s=3300.,method='Radau'):
        self.alpha0 = alpha0
        self.N = len(alpha0)
        self.D = D
        self.D0 = np.array([D[i](0.) for i in range(self.N)])
        self.lambdas = lambdas
        self.F  = F
        self.dFdz = dFdz
        self.phi = phi
        self.rho_f = rho_f
        self.rho_s = rho_s
        self.method = method
    
    def F_bar(self,zp):
        '''
        returns  numpy array of  size (len(zp),len(D)) for
        
        Fbar_D = F + (1. - F)*D_i
        '''
        D = self.D
        F = self.F(zp)
        if np.isscalar(zp):
            F_bar_D = np.array([ F + (1. - F)*D[i](zp) for i in range(len(D))])
        else :
            F_bar_D = np.zeros((len(zp),len(D)))
            F_bar_D = np.array([ F + (1. - F)*D[i](zp) for i in range(len(D))]).T
        return F_bar_D
    
    def rho_bar(self,zp):
        '''
        returns numpy array of  size (len(zp),len(D)) for
        
        rho_bar_D = rho_f/rho_s*phi + (1. - phi)*D_i
        '''
        rho_s = self.rho_s
        rho_f = self.rho_f
        
        phi = self.phi(zp)
        D = self.D
        if np.isscalar(zp):
            rho_bar_D = np.array([ rho_f/rho_s*phi + (1. - phi)*D[i](zp) for i in range(len(D))])
        else: 
            rho_bar_D = np.zeros((len(zp),len(D)))
            rho_bar_D = np.array([ rho_f/rho_s*phi + (1. - phi)*D[i](zp) for i in range(len(D))]).T
           
        return rho_bar_D
    
    def rhs(self,z,Ur):
        '''
        Returns right hand side of generic decay chain problem for the log of the radiogenic concentration.
        
        The full equation for dU/dz is given by Eq (9) in Spiegelman 2000, but here we split
        
        Uf = U^st + U^r where
        U^st is the log of the stable element concentrations U^s = log(D(0)/Fbar_z)
        U^r is the radiogenic ingrowth part
        
        Solid concentration is ignored as unnecessary in this version, but would be simply the fluid
        concentration times the partition coefficient for that depth.
        
        The general equation is 
            dU_i^r/dz = h\lambda_i/Weff_i * [ R_i^{i-1} exp(Uf_{i-1} - Uf_i) - 1.)
            
        This routine assumes that lambda, D, D0, lambda_tmp, phi0, W_0 and alpha_0 are set by the UserCalc driver routine.
        '''
            
        # determine F_bar(z) and rho_bar(z) once
        Fb = self.F_bar(z)
        rb = self.rho_bar(z)
        
        # initial value of partition coefficients
        D0 = self.D0
        
        # initial value densities
        rho_f = self.rho_f
        rho_s = self.rho_s
        
        # stable concentration
        Ust = np.log(D0/Fb)
        
        # total melt concentration
        Uf = Ust + Ur

        # effective velocity and scaled rate term
        lambda_prime = self.lambdas*rb/Fb
        
        # ingrowth factor and exponential factor
        R = np.zeros(len(lambda_prime))
        expU = np.zeros(len(lambda_prime))
        for i in range(1,len(lambda_prime)):
            R[i] = self.alpha0[i]*D0[i]/D0[i-1]*rb[i-1]/rb[i]
            expU[i] = np.exp(Uf[i-1]-Uf[i])
            
        # return full RHS
        
        return lambda_prime*(R*expU - 1.)
        
    def solve(self,z_eval=None):
        '''
        solves generic radioactive decay chain problem as an ODE initial value problem
        if z_eval = None, save every point
        else save output at every z_eval depth
        '''
        
        # Set initial condition and solve ODE      
        Ur_0 = np.zeros(len(self.D0))
        sol = solve_ivp(self.rhs,(0.,1.),Ur_0,t_eval=z_eval,method=self.method)
        z = sol.t
        Ur = sol.y
        # calculate stable component of fluid concentration
        Ust = np.log(self.D0/self.F_bar(z)).T 
        # calculate total fluid concentration
        Uf = Ur + Ust
        # placeholder for solid concentration (not used here, but cannot be blank)
        Us = Uf
        return z,Us,Uf
        
        

## A disequilibrium decay chain solver class

This class solves the disequilibrium transport problem described in Spiegelman and Elliott, 1993, i.e., Eqs. (26) and (27) rewritten in terms of the logs of the concentrations:

$$
    U^s_i = \log\left(\frac{c_i^{s}}{c_{i,0}^s}\right),  \quad U^f_i = \log\left(\frac{c_i^{f}}{c_{i,0}^f}\right) 
$$
thus
$$
    \frac{dU_i}{dz} = \frac{1}{c_i} \frac{dc_i}{dz}
$$

In [ ]:
# This is the equilibrium transport calculator. You probably will not need or want to edit this cell.
# A possible exception: the ODE solver method can be changed if desired, but the default is probably best.

from scipy.integrate import solve_ivp

class DisequilTransport:
    '''
    A class for calculating generic radioactive decay chains for the scaled equations 9 in Spiegelman (2000)
    
    Usage:  solver=model(alpha0,lambdas,D,F,dFdz,phi,rho_f=2800., rho_s=3300.,method='Radau')
    
    inputs:
        alpha0  :  numpy array of initial activities
        lambdas :  decay constants scaled by solid transport time
        D       :  Function returning an array of partition coefficents at scaled height z'
        F       :  Function that returns the degree of melting F as a function of  z'
        dFdz    :  Function that returns the derivative of F with respect to z'
        phi     :  Function that returns the porosity phi as a function of z'
        rho_f   :  melt_density
        rho_s   :  solid_density
        method  :  ODE solver method to be passed to ode_solveivp (one of 'RK45', 'Radau', 'BDF')
        
    Outputs:  pandas DataFrame with columns z, Us, Uf
    '''
    def __init__(self,alpha0,lambdas,D,F,dFdz,phi,rho_f=2800.,rho_s=3300.,method='Radau'):
        self.alpha0 = alpha0
        self.N = len(alpha0)
        self.D = lambda zp: np.array([D[i](zp) for i in range(self.N) ])
        self.D0 = self.D(0.)
        self.lambdas = lambdas
        self.F  = F
        self.dFdz = dFdz
        self.phi = phi
        self.rho_f = rho_f
        self.rho_s = rho_s
        self.method = method
        
    def rhs(self,z,U):
        '''
        Returns right hand side of generic chain problem for the log of concentration of the solid
        U^s = U[:N]  where N=length of the decay chain
        and
        U^f = U[N:]
        
        The full equation for dU/dz is given by Eqs 28 and 29 in Spiegelman and Elliott (1993), but
        in terms of concentrations c^s and c^f. Here log concentrations are calculated as in Spiegelman (2000).
            
        This routine assumes that lambda, D, D0, lambda_tmp, phi0, W_0 and alpha_0 are set by the UserCalc driver routine.
        '''
        
        # calculate F(z) and D(z)
        F = self.F(z)
        dFdz = self.dFdz(z)
        D = np.array(self.D(z))
        phi = self.phi(z)
        
        # initial value of partition coefficients
        D0 = self.D0
        
        # split U into solid and melt components for convenience
        N = self.N
        Us = U[:N]
        Uf = U[N:]
        
        # stable melting component of dU
        dUs = (1. - 1./D)/(1.-F)*dFdz
        # careful here about the initial gradient (and floating point test)
        if F == 0.:
            dUf = dUs/2.
        else:
            dUf = (D0/D*np.exp(Us - Uf) -1.)/F
            
        # ingrowth terms
        Rs = np.zeros(N)
        Rf = np.zeros( N)
        a0 = self.alpha0
        for i in range(1,N):
            Rs[i] = a0[i-1]/a0[i]*np.exp(Us[i-1]-Us[i])
            Rf[i] = (D0[i]*a0[i-1])/(D0[i-1]*a0[i])*np.exp(Uf[i-1]- Uf[i])
            
        # add to stable terms
        dUs += (1 - phi)/(1. - F)*self.lambdas*(Rs - 1.)
        if F == 0.:
            dUf +=  self.lambdas*(Rf - 1.)
        else:
            dUf += (self.rho_f*phi)/(self.rho_s*F)*self.lambdas*(Rf - 1.)
            
        # return full RHS
        dU=np.zeros(2*N)
        dU[:N] = dUs
        dU[N:] = dUf
        
        return dU
        
    def solve(self,z_eval=None):
        '''
        solves generic radioactive decay chain problem as an ODE initial value problem
        if z_eval = None,  save every point
        else save output at every z_eval depth
        '''
        
        # Set initial condition and solve ODE  
        N = self.N
        U0 = np.zeros(2*N)
        try:
            sol = solve_ivp(self.rhs,(0.,1.),U0,t_eval=z_eval,method=self.method)
            z = sol.t
            U = sol.y
        except ValueError as err:
            print('Warning:  solver did not complete, returning NaN: {}'.format(err))
            z = np.array([1.])
            U = np.NaN*np.ones((2*N,len(z)))
        # return solid and liquid concentrations
        Us = U[:N,:]
        Uf = U[N:,:]
        return z,Us,Uf
        

## The main UserCalc class

This class takes in a dataframe of inputs, initializes the appropriate splines and porosity functions, and
then presents a set of methods for calculating 1D columns and grid figures.

In [ ]:
# This is the UserCalc driver routine. This should not be edited or changed.

from scipy.optimize import brentq
from scipy.interpolate import interp1d, pchip

class UserCalc:
    ''' A class for constructing solutions for Equilibrium Transport U-series calculations
        ala Spiegelman, 2000, G-cubed
        
        Usage: 
        us = UserCalc(model,df,dPdz = 0.32373, n = 2., tol=1.e-6, phi0 = 0.008, W0 = 3.) 
        
        with 
        model : a class of a Useries decaychain model
        df   :  a pandas-dataframe with columns ['P','Kr','DU','DTh','DRa','DPa']
        dPdz :  Pressure gradient to convert P to z
        n    :  permeability exponent
        tol  :  tolerance for ODE solver
        phi0 :  initial porosity
        W0   : upwelling velocity (cm/yr)
            
    '''
    def __init__(self, model, df, dPdz = 0.32373, n = 2., tol = 1.e-6, phi0 = 0.008, W0 = 3.):
        self.model = model
        self.df = df
        self.dPdz = dPdz
        self.n = n
        self.tol = 1.e-6
        self.phi0 = phi0
        self.W0 = W0/1.e5
        
        # set depth scale h
        self.zmin = df['P'].min()/dPdz
        self.zmax = df['P'].max()/dPdz
        self.h = self.zmax - self.zmin
        
        # lambda function to define scaled column height zprime
        self.zp = lambda P: (self.zmax - P/dPdz)/self.h
        
        # set interpolants for F and Kr and pressure
        self.F = pchip(self.zp(df['P']),df['F'])
        self.dFdz = self.F.derivative(nu=1)
        self.Kr = interp1d(self.zp(df['P']),df['Kr'],kind='cubic')
        self.P = interp1d(self.zp(df['P']),df['P'],kind='cubic')

        # set maximum degree of melting
        self.Fmax = self.df['F'].max()
        
        # set reference densities (assuming a mantle composition)
        self.rho_s = 3300.
        self.rho_f = 2800.
        
        # set  decay constants for [ 238U, 230Th, 226Ra] and [ 235U, 231Pa ]       
        t_half_238 = np.array([4.468e9, 7.54e4, 1600.])
        t_half_235 = np.array([7.03e8, 3.276e4])
        self.lambdas_238 = np.log(2.)/t_half_238
        self.lambdas_235 = np.log(2.)/t_half_235
        
        # set interpolation functions for Partition coefficients for each chain
        self.D_238 = [ interp1d(self.zp(df['P']),df['DU'],kind='cubic'),
                       interp1d(self.zp(df['P']),df['DTh'],kind='cubic'),
                       interp1d(self.zp(df['P']),df['DRa'],kind='cubic') ]
        self.D_235 = [ interp1d(self.zp(df['P']),df['DU'],kind='cubic'),
                       interp1d(self.zp(df['P']),df['DPa'],kind='cubic')]
        
        # lambda function to get partition coefficients at zprime = 0
        self.get_D0 = lambda D: np.array([ D[i](0) for i in range(len(D))])
                        
        # initialize reference permeability
        self.setAd(self.phi0,n=n)
        
    # initialize porosity function    
    def setAd(self,phi0,n):
        ''' 
            sets the reference permeability given the maximum porosity 
        '''
        Fmax = self.Fmax
        self.phi0 = phi0
        self.n = n
        self.Ad =  (self.rho_s/self.rho_f*Fmax - phi0*(1. - Fmax)/(1. - phi0))/(phi0**n*(1.-phi0))
        
    def phi(self,zp):
        '''
        returns porosity as function of dimensionless column height zp
        '''
        # effective permeability
        K = self.Kr(zp)*self.Ad
        
        # degree of melting
        F = self.F(zp)
        
        # density ratio
        rs_rf = self.rho_s/self.rho_f
        
        # rootfinding function to define porosity such that f(phi) = 0
        
        # check if scalar else loop
        if np.isscalar(zp):
            f = lambda phi: K*phi**self.n*(1. - phi)**2 + phi*(1. + F*(rs_rf - 1.)) - F*rs_rf
            upper_bracket= 1.05*self.phi0
            try:
                phi = brentq(f,0.,upper_bracket)
            except ValueError:
                phi_test = np.linspace(0,upper_bracket)
                print('Error in brentq: brackets={}, {}'.format(f(0.),f(upper_bracket)))
                print('zp={},F={}, K={}'.format(zp,F,K))                    
        else: # loop over length of zp
            phi = np.zeros(zp.shape)
            for i,z in enumerate(zp):
                f = lambda phi: K[i]*phi**self.n*(1. - phi)**2 + phi*(1. + F[i]*(rs_rf - 1.)) - F[i]*rs_rf
                phi[i] = brentq(f,0.,1.)
        return phi
    
    def set_column_params(self, phi0, n, W0):
        '''
        set porosity/permeability and upwelling rate parameters for a single column
        
        phi0: porosity at Fmax
        n   : permeability exponent
        W0  : upwelling rate (cm/yr)
        '''
    
        self.setAd(phi0,n)
        self.W0 = W0/1.e5  # upwelling in km/yr.
        
    def solve_1D(self,D,lambdas,alpha0 = None, z_eval = None):
        '''
        Solves 1-D decay problem (assumes column parameters have been set
        
        Usage:  z, a, Us, Uf = solve_1D(D,lambdas,alpha0,z_eval)
        
        Input:
        D      :  function that returns bulk partition coefficients for each nuclide
        lambdas:  decay coefficients of each nuclide
        alpha0 :  initial activities of the nuclide in the unmelted solid (defaults to 1)
        z_eval :  dimensionless column heights where solution is returned
        
        Output: 
        z:   coordinates where evaluated
        a:   activities of each nuclide
        Us:  solid nuclide concentration
        Uf:  fluid nuclide concentration
        '''
        
        # if z_eval is not set, use initial Input values
        if z_eval is None:
            z_eval = self.zp(self.df['P'])
        elif np.isscalar(z_eval):
            z_eval = np.array([z_eval])
            
        # if alpha is not set, use 1
        if alpha0 is None:
            alpha0 = np.ones(len(lambdas))
        
        # scaled decay constants and initial partition coefficients    
        lambdap = self.h*lambdas/self.W0
    
        us = self.model(alpha0,lambdap,D,self.F,self.dFdz,self.phi,self.rho_f,self.rho_s)
        
        D0 = self.get_D0(D)
        z, Us, Uf = us.solve(z_eval)
        
        # calculate activities
        act =  [ alpha0[i]/D0[i]*np.exp(Uf[i]) for i in range(len(D0)) ]
        return z, act, Us, Uf        
        
    def solve_all_1D(self,phi0 ,n , W0, alphas = np.ones(4), z_eval = None):
        '''
        Sets up and solves the 1-D column model for a given phi0,n, and upwelling rate W0 (in cm/yr).
        Solves for both the U238 decay chain and the U235 decay chain.
        
        Returns a pandas dataframe
        '''
        self.set_column_params(phi0,n,W0)
        
        # evaluate at input depths if not specified
        if z_eval is None:
            z_eval = self.zp(self.df['P'])
                    
        # solve for the U238 model
        z238, a238, Us238, Uf238 = self.solve_1D(self.D_238, self.lambdas_238, z_eval = z_eval)
        
        # solve for the U235 model
        z235, a235, Us235, Uf235 = self.solve_1D(self.D_235, self.lambdas_235, z_eval = z_eval)
        
        # start building output dataframe
        z = z_eval
        
        df = pd.DataFrame()
        df['P'] = self.P(z) 
        df['z'] = self.zmax - self.h*z
        df['F'] = self.F(z)
        df['phi'] = self.phi(z)
        names = ['(230Th/238U)','(226Ra/230Th)']
        for i,name in enumerate(names):
            df[name] = a238[i+1]/a238[i]
        
        df['(231Pa/235U)'] = a235[1]/a235[0]
        
        names = ['Uf_238U','Uf_230Th', 'Uf_226Ra']
        for i,name in enumerate(names):
            df[name] = Uf238[i] 
            
        names = ['Us_238U','Us_230Th', 'Us_226Ra']
        for i,name in enumerate(names):
            df[name] = Us238[i] 
            
        names = ['Uf_235U','Uf_231Pa']
        for i,name in enumerate(names):
            df[name] = Uf235[i]
            
        names = ['Us_235U','Us_231Pa']
        for i,name in enumerate(names):
            df[name] = Us235[i]
            
        return df
    
    def solve_grid(self, phi0, n, W0, D, lambdas, alpha0 = None, z = 1.):
        '''
        solves of activity ratios at the height z in the column for a mesh grid of porosites phi0 and upwelling
        velocities W0 (slow, not vectorized)
        
        '''
        # number of nuclides in chain
        Nchain = len(lambdas)
        
        # if alpha is not set, use 1
        if alpha0 is None:
            alpha0 = np.ones(Nchain)
            
        act = np.zeros((Nchain - 1,len(W0),len(phi0)))
                       
        for j, W in enumerate(W0):
            print('\nW = {}'.format(W), end=" ")
            for i, phi in enumerate(phi0):
                print('.', end=" ")
                self.set_column_params(phi,n,W)
                z, a, Us, Uf = self.solve_1D(D,lambdas,alpha0,z_eval = z)
                for k in range(1,Nchain):
                    act[k-1,j,i] = a[k]/a[k-1]
        
        return act
    

## Look at input data

In [ ]:
input_file = 'input_files/{}.csv'.format(runname)
df = pd.read_csv(input_file,skiprows=1,dtype=float)
df = df[df.P > Plithos]

In [ ]:
plot_inputs(df)

In [ ]:
df

### Solve the 1-D column problem

In [ ]:
# Initialize the solver object

# Choose and activate the model desired in the row below. The 'EquilTransport' does a full equilibrium
# (i.e., reactive porous flow) calculation, while 'DisequilTransport' determines full disequilibrium.

us = UserCalc(EquilTransport,df)
#us = UserCalc(DisequilTransport,df)


# Set desired parameters for initial model run. Note that if nothing is entered here, the model default assumes
# phi0 = 0.008, W0 = 3 cm/yr., n = 2, rho_f = 2800, and rho_s = 3300.

phi0 = 0.008
W0 = 3. # cm/yr.
n = 2.
rho_f = 2800.
rho_s = 3300.

df_out = us.solve_all_1D(phi0,n,W0)

## Activities at the top of the column

In [ ]:
df_out[['(230Th/238U)','(226Ra/230Th)','(231Pa/235U)']].iloc[-1]

In [ ]:
df_out

## Plot solution with depth

In [ ]:
plot_1Dcolumn(df_out)
plt.show()

## View the dataframe

In [ ]:
df_out

### Save the dataframe as a csv file

In [ ]:
filename='sample_out.csv'
df_out.to_csv(filename)

## Calculated gridded activity ratios at the top of the column

In [ ]:
# Activate and edit the rows below to set phi and W values at evenly spaced log grid intervals:

# phi0 = np.logspace(-3,-2,11)
# W0 = np.logspace(-1,1,11)

# Activate and edit the rows below to set specific desired phi and W values for the grid:
phi0 = np.array([0.001, 0.002, 0.005, 0.01])
W0 = np.array([0.5, 1., 2., 5., 10., 20., 50.])  # cm/yr.

import time
tic = time.clock()
toc = time.clock()
print('\nelapsed time={}'.format(toc-tic))

# Calculate the U-238 decay chain activity ratios for final melts generated at all points in W vs. phi grid space:
act = us.solve_grid(phi0, n, W0, us.D_238, us.lambdas_238 )

In [ ]:
# Save the U-238 decay chain grid results as .csv files.

Th = act[0]
Ra = act[1]

df = pd.DataFrame(Th)
df.to_csv("Th_grid_{}.csv".format(runname))
df = pd.DataFrame(Ra)
df.to_csv("Ra_grid_{}.csv".format(runname))

In [ ]:
# Calculate the U-235 decay chain activity ratios for final melts generated at all points in W vs. phi grid space:
act_235 = us.solve_grid(phi0, n, W0, us.D_235, us.lambdas_235 )
Pa = act_235[0]

# Save the U-235 decay chain grid results as a .csv file.
df = pd.DataFrame(Pa)
df.to_csv("Pa_grid_{}.csv".format(runname))

## Plot gridded decay results using W vs. phi contour plots

In [ ]:
plot_contours(phi0,W0,act,figsize=(12,12))

In [ ]:
plot_contours(phi0,W0,act_235,figsize=(8,8))

## Plot gridded decay results using activity ratio-activity ratio plots

In [ ]:
plot_mesh_Ra(Th,Ra,figsize=(8,8))

In [ ]:
plot_mesh_Pa(Th,Pa,figsize=(8,8))